In [23]:
#the training code for recommender systems has been taken from Tutorial adapted from 
# content originally created for IVADO's Workshop on Recommender Systems, August 2019.
#credits Dora Jambor, David Beiber, Laurent Charlin

import pandas as pd
import numpy as np

df= pd.read_csv('./ghazal/df_matrix')

In [14]:
#random filling user data for now.
#but lets say we collect this data from a google sheet every few days
# or through a google forms poll
df_ratings=df.copy()
for i in range(1,df.shape[1]):
    df_ratings[str(i)]= np.random.randint(0, 4, df.shape[0])

df_ratings= df_ratings.T
print(df_ratings)
df_ratings.to_csv('./ghazal/random_user_item_data_test',index=False)

                 0           1           2                 3   \
author  ahmad-faraz  meer-anees  nida-fazli  akbar-allahabadi   
1                 3           1           3                 0   
2                 2           3           0                 2   
3                 0           0           3                 1   
4                 2           2           2                 3   
5                 2           2           0                 3   
6                 1           0           3                 1   

                      4               5                   6                7   \
author  jigar-moradabadi  nazm-tabatabai  wali-mohammad-wali  sahir-ludhianvi   
1                      0               3                   1                0   
2                      3               1                   2                2   
3                      0               2                   1                3   
4                      0               1                   0              

In [15]:
df_ratings=pd.read_csv('./ghazal/random_user_item_data_test')


df_ratings.columns = df_ratings.iloc[0]
df_ratings = df_ratings[1:]
a= df_ratings.to_numpy()

print(a.shape)
a= a.astype(int)
print(type(a[0][0]))

(6, 29)
<class 'numpy.int64'>


In [16]:
content_info_df= df.copy()
del content_info_df['author']

content_info= content_info_df.to_numpy()
print(content_info.shape)

(29, 6)


In [17]:

def learn_to_recommend(data,content_features, features=10, lr=0.0002, epochs=101, weigth_decay=0.02, stopping=0.001):
    """
    Args:
       data: every evaluation
       features: number of latent variables
       lr: learning rate for gradient descent
       epochs: number of iterations or maximum loops to perform
       weigth_decay: L2 regularization to predict rattings different of 0
       stopping: scalar associated with the stopping criterion
      
     Returns:
       P: latent matrix of users
       Q: latent matrix of items
       loss_train: vector of the different values of the loss function after each iteration on the train
       loss_valid: vector of the different values of the loss function after each iteration not on valid
       """
     
    train = data
    nb_users, nb_items = len(train), len(train[0])

    loss_train = []

    P = np.random.rand(nb_users, features) * 0.1
    Q =  content_features #np.random.rand(nb_items, features) * 0.1 +
    print('shape of Q inside learning to recommend: ', Q.shape)
    print('shape of content features: ', content_features.shape)
    for e in range(epochs):        
        for u in range(nb_users):
            for i in range(nb_items):

                # Answer 4.1: Code the condition
                if train[u][i] > 0:
                    error_ui = train[u][i] - prediction(P, Q, u, i)
                    P, Q = sgd(error_ui, P, Q, u, i, features, lr, weigth_decay)
                               
       
        loss_train.append(loss(train, P, Q))
        
        
        if e % 10 == 0:
            print('Epoch : ', "{:3.0f}".format(e+1), ' | Train :', "{:3.3f}".format(loss_train[-1]))
            
        
    return P, Q, loss_train

In [18]:

def prediction(P, Q, u, i):
    """
    Args:
        P: user matrix
        Q: matrix of items
        Q is item matrix here
        u: index associated with user u
        i: index associated with item i
    Returns:
        pred: the predicted evaluation of the user u for the item i
    """
    
    return np.dot(P[u,:], Q[i,:])

def loss(data, P, Q):
    """
    Args:
       data: ratings
       P: matrix of users
       Q: matrix of items   
    Returns:
        MSE: observed mean of squared errors 
    """
    errors_sum, nb_evaluations = 0., 0
    nb_users, nb_items = len(data), len(data[0])

    for u in range(nb_users):
        for i in range(nb_items):
        
            # Answer 5.2:
            if data[u][i] > 0:
                errors_sum += pow(data[u][i] - prediction(P, Q, u, i), 2)
                nb_evaluations += 1
                
    return errors_sum / nb_evaluations

In [19]:
def sgd(error, P, Q, id_user, id_item, features, lr, weigth_decay):
    """
    Args:
        error: difference between observed and predicted evaluation (in that order)
        P: matrix of users
        Q: matrix of items
        id_user: id_user
        id_item: id_item
        features: number of latent variables
        lr: learning for the descent of the gradient
        weigth_decay: scalar multiplier controlling the influence of the regularization term
       
     Returns:
        P: the new estimate for P
        Q: the new estimate for Q
        Q is item matrix here
     """    
    
    
   
    for f in range(features):
        P[id_user, f] = P[id_user, f] + lr * (2 * Q[id_item, f] * error - 2 * weigth_decay * P[id_user, f])
        Q[id_item, f] = Q[id_item, f] + lr * (2 * P[id_user, f] * error - 2 * weigth_decay * Q[id_item, f])
      
    return P, Q


In [20]:
# setting features as 6, that is k=6. lets imagine that dimension
# of latent vector works fine , as we also had 6 dim topics in our content
features = 6
lr = 0.01
epochs = 101
weigth_decay = 0.02
stopping = 0.001
train=a
P, Q, loss_train = learn_to_recommend(train,content_info, features, lr, epochs, weigth_decay, stopping)

shape of Q inside learning to recommend:  (29, 6)
shape of content features:  (29, 6)
Epoch :    1  | Train : 4.143
Epoch :   11  | Train : 0.454
Epoch :   21  | Train : 0.390
Epoch :   31  | Train : 0.257
Epoch :   41  | Train : 0.127
Epoch :   51  | Train : 0.055
Epoch :   61  | Train : 0.025
Epoch :   71  | Train : 0.014
Epoch :   81  | Train : 0.009
Epoch :   91  | Train : 0.007
Epoch :  101  | Train : 0.006


In [21]:
print('User matrix shape: ',P.shape)
print('Item matrix shape: ',Q.shape)

User matrix shape:  (6, 6)
Item matrix shape:  (29, 6)


In [22]:
#lets say a new user u comes. 
#we can assume asking choices on some of the items
# then model this new user to some closest user we already have 
#and use that to recommend

#lets say we know he likes these 3 items bsaed on a question prompt
new_user=np.zeros((29,1))
new_user[2]=1
new_user[8]=1
new_user[12]=1


def new_user_most_similar(new_user,r):
    scores= np.dot(r,new_user).reshape(r.shape[0])
    
    indices= scores.argsort()[-1:][::-1]
    return indices
sim_user_index=new_user_most_similar(new_user,a)


#get score with all items and return highest 3 indices
def get_top_indices(Q,sim_inx):
    
    scores= np.dot(P[sim_inx],Q.T).reshape(Q.shape[0])
    indices= scores.argsort()[-3:][::-1]
    return indices

def get_top_items(idxs,df):
    items=[]
    for idx in idxs:
        items.append(df.iloc[idx][0])
    return items

indices=get_top_indices(Q,sim_user_index)
items=get_top_items(indices,df)
print(items)

['sahir-ludhianvi', 'allama-iqbal', 'nida-fazli']


In [39]:
#keep generating new users.
new_user=np.zeros((29,1))
choose=np.random.randint(0, 4,5)
for c in choose:
    new_user[c]=1
print(new_user.reshape(new_user.shape[0]))    

[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
